Part 2 of the lab : adding latitude and longitude for each postal code

In [35]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)

cols=['PostalCode', 'Borough','Neighborhood']
df = pd.DataFrame(columns=cols)
df.set_index('PostalCode')

soup = BeautifulSoup(r.text, 'lxml')
table = soup.find_all('table')[0]

for index, row in enumerate(table.find_all('tr'), start=0):
        columns = row.find_all('td')
        datax = list()
        for col_index, column in enumerate(columns,start=0):
                datax.append(column.get_text().strip())
        
        if len(datax) == 3:
                if datax[1] != 'Not assigned':
                    if datax[2] == 'Not assigned':
                        datax[2] = datax[1]
                    df = df.append({'PostalCode': datax[0],
                  'Borough': datax[1],'Neighborhood': datax[2]}, ignore_index=True)
        
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].agg(lambda x: ','.join(x)).to_frame().reset_index()

    


In [36]:
geodata = pd.read_csv("http://cocl.us/Geospatial_data")

geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [37]:
geodata.rename(columns={"Postal Code": "PostalCode"},inplace=True)

In [38]:
df['Latitude'] = 0
df['Longitude'] = 0

In [39]:
result = pd.merge(df, geodata, how="left", on=["PostalCode"])
result.head()

,PostalCode,Borough,Neighborhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M1B,Scarborough,"Rouge,Malvern",0,0,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",0,0,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",0,0,43.763573,-79.188711
3,M1G,Scarborough,Woburn,0,0,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,0,0,43.773136,-79.239476


In [40]:
result.drop("Latitude_x",axis=1, inplace=True)
result.drop("Longitude_x",axis=1, inplace=True)
result.rename(columns={"Latitude_y": "Latitude", "Longitude_y":"Longitude"},inplace=True)
result.head()



,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
